In [10]:
!pip install geopandas
import geopandas as gpd
import pandas as pd


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [108]:
usagers = pd.read_csv('/Users/Pierr/Downloads/usagers-2021.csv', sep=";")
vehicules = pd.read_csv('/Users/Pierr/Downloads/vehicules-2021.csv', sep=";")

In [110]:
usagers.head()

,Num_Acc,id_vehicule,num_veh,place,catu,grav,sexe,an_nais,trajet,secu1,secu2,secu3,locp,actp,etatp
0,202100000001,201 764,B01,1,1,3,1,2000.0,1,0,9,-1,0,0,-1
1,202100000001,201 765,A01,1,1,1,1,1978.0,1,1,-1,-1,0,0,-1
2,202100000002,201 762,A01,1,1,4,1,1983.0,0,1,-1,-1,0,0,-1
3,202100000002,201 763,B01,1,1,3,1,1993.0,0,1,-1,-1,0,0,-1
4,202100000003,201 761,A01,1,1,1,1,1995.0,1,1,0,-1,0,0,-1


In [111]:
#to drop : id_vehicule, num_vehicule, place, secu1, secu2, secu3
usagers = usagers.drop(columns=['id_vehicule', 'num_veh', 'place', 'secu1', 'secu2', 'secu3'])

In [112]:
vehicules.head()

,Num_Acc,id_vehicule,num_veh,senc,catv,obs,obsm,choc,manv,motor,occutc
0,202100000001,201 764,B01,1,1,0,2,1,1,5,NaN
1,202100000001,201 765,A01,1,7,0,9,3,17,1,NaN
2,202100000002,201 762,A01,0,7,2,2,1,1,0,NaN
3,202100000002,201 763,B01,0,7,0,2,1,9,0,NaN
4,202100000003,201 761,A01,1,7,0,1,3,1,1,NaN


In [ ]:
vehicules = vehicules.drop(columns=['id_vehicule', 'num_veh','senc', 'manv', 'motor','occutc'])

In [137]:
df = usagers.merge(vehicules,on=['Num_Acc'])
df.head()

,Num_Acc,catu,grav,sexe,an_nais,trajet,locp,actp,etatp,catv,obs,obsm,choc
0,202100000001,1,3,1,2000.0,1,0,0,-1,1,0,2,1
1,202100000001,1,3,1,2000.0,1,0,0,-1,7,0,9,3
2,202100000001,1,1,1,1978.0,1,0,0,-1,1,0,2,1
3,202100000001,1,1,1,1978.0,1,0,0,-1,7,0,9,3
4,202100000002,1,4,1,1983.0,0,0,0,-1,7,2,2,1


In [80]:
#on ne garde dans le dataframe que les lignes implicant un accident entre un vélo et
#un autre usager ou catv == 1 représente les vélos
Num_Acc =  df[df["catv"] == 1]["Num_Acc"]
velo_only = df[df["Num_Acc"].isin(list(Num_Acc))]


In [82]:
velo_only.head()

,Num_Acc,id_vehicule,num_veh,senc,catv,obs,obsm,choc,manv,motor,occutc,catu,grav,sexe,an_nais,trajet,locp,actp,etatp
0,202100000001,201 764,B01,1,1,0,2,1,1,5,NaN,1,3,1,2000.0,1,0,0,-1
1,202100000001,201 764,B01,1,1,0,2,1,1,5,NaN,1,1,1,1978.0,1,0,0,-1
2,202100000001,201 765,A01,1,7,0,9,3,17,1,NaN,1,3,1,2000.0,1,0,0,-1
3,202100000001,201 765,A01,1,7,0,9,3,17,1,NaN,1,1,1,1978.0,1,0,0,-1
18,202100000006,201 752,B01,2,1,0,0,5,1,5,NaN,1,4,1,2009.0,2,0,0,-1


In [51]:
usagers_velo.describe()

,Num_Acc,catu,grav,sexe,an_nais,trajet,locp,etatp
count,1.065100e+04,10651.000000,10651.000000,10651.000000,10255.000000,10651.000000,10651.000000,10651.000000
mean,2.021000e+11,1.092667,2.451882,1.191719,1978.862311,3.356305,-0.375645,-0.929678
std,1.535996e+04,0.381440,1.394021,0.614879,19.424436,2.862299,0.890958,0.410236
min,2.021000e+11,1.000000,-1.000000,-1.000000,1924.000000,-1.000000,-1.000000,-1.000000
25%,2.021000e+11,1.000000,1.000000,1.000000,1964.000000,1.000000,-1.000000,-1.000000
50%,2.021000e+11,1.000000,3.000000,1.000000,1981.000000,4.000000,0.000000,-1.000000
75%,2.021000e+11,1.000000,4.000000,2.000000,1995.000000,5.000000,0.000000,-1.000000
max,2.021001e+11,3.000000,4.000000,2.000000,2021.000000,9.000000,9.000000,3.000000
